In [1]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import whisper
import torchaudio

from tqdm.notebook import tqdm

from whisper import audio


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

c:\Users\mailt\anaconda3\lib\site-packages\torchaudio\backend\utils.py:67: UserWarning: No audio backend is available.
  warnings.warn('No audio backend is available.')


In [8]:
def exact_div(x, y):
    assert x % y == 0
    return x // y


In [12]:
SAMPLE_RATE = 16000
N_FFT = 400
N_MELS = 80
HOP_LENGTH = 160
CHUNK_LENGTH = 30
N_SAMPLES = CHUNK_LENGTH * SAMPLE_RATE  # 480000: number of samples in a chunk
N_FRAMES = exact_div(N_SAMPLES, HOP_LENGTH)

In [2]:
model = whisper.load_model("tiny.en")
print(
    f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters."
)

Model is English-only and has 37,184,256 parameters.


In [3]:
audio = "data_bbc.mp3"

In [4]:
mel = whisper.log_mel_spectrogram(audio)

In [7]:
mel[:, 0:].shape

torch.Size([80, 16551])

In [8]:
decode_options = {}
decode_options["language"] = "en"

In [10]:
language = decode_options["language"]
task = decode_options.get("task", "transcribe")
tokenizer = whisper.tokenizer.get_tokenizer(model.is_multilingual, language=language, task=task)

In [13]:
pad_or_trim(mel[:, seek:], N_FRAMES)

tensor([-0.7150, -0.7150, -0.7150,  ...,  1.0362,  1.0810,  0.0587])

In [14]:
model.embed_audio(mel[0])

RuntimeError: Expected 3-dimensional input for 3-dimensional weight [384, 80, 3], but got 1-dimensional input of size [16551] instead